In [202]:
from nltk.corpus import stopwords
import string
from nltk.tokenize import RegexpTokenizer
import spacy
nlp = spacy.load("ru_core_news_sm")
import gensim.downloader
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import random
from gensim import models
from collections import Counter

In [2]:
tokenizer = RegexpTokenizer('\w+')

In [3]:
def lemmatize(text):
    while '»' in text:
        text = text.replace('»', '')
    while '«' in text:
        text = text.replace('«', '')
    while '(' in text:
        text = text.replace('(', '')
    while ')' in text:
        text = text.replace(')', '')

    symbs = ['—', "»", '«', 'note', '…']
    stop = set(stopwords.words('russian') + list(string.punctuation) + symbs)

    doc = nlp(text) 

    all_sentences = []
    for sent in doc.sents:
        tokens = [i for i in tokenizer.tokenize(str(sent).lower()) if i not in stop]

        lemmatized = []
        for word in tokens:
            lemma = morph.parse(word)[0].normal_form
            lemmatized.append(lemma)
        all_sentences.append(lemmatized)

    return all_sentences

In [4]:
with open('otcy_i_deti_1862.txt', 'r') as file:
    text = file.read()
words = lemmatize(text)

In [51]:
model = gensim.models.Word2Vec(
    window = 5,
    min_count=2,
    workers = 4
    )

In [52]:
model.build_vocab(words, progress_per=100)

In [54]:
model.corpus_count

4105

In [55]:
model.train(words, total_examples=model.corpus_count, epochs = model.epochs)

(133424, 163585)

In [56]:
model.save('testmodel')

In [201]:
for index, word in enumerate(model.wv.index_to_key):
    print(f"word #{index}/{len(model.wv.index_to_key )} is {word}")

word #0/3576 is это
word #1/3576 is базар
word #2/3576 is петрович
word #3/3576 is аркадий
word #4/3576 is свой
word #5/3576 is который
word #6/3576 is павел
word #7/3576 is человек
word #8/3576 is говорить
word #9/3576 is николай
word #10/3576 is сказать
word #11/3576 is рука
word #12/3576 is знать
word #13/3576 is ты
word #14/3576 is сергеевич
word #15/3576 is промолвить
word #16/3576 is базаров
word #17/3576 is сам
word #18/3576 is глаз
word #19/3576 is анна
word #20/3576 is мочь
word #21/3576 is один
word #22/3576 is иванович
word #23/3576 is василий
word #24/3576 is какой
word #25/3576 is спросить
word #26/3576 is лицо
word #27/3576 is одинцов
word #28/3576 is очень
word #29/3576 is голова
word #30/3576 is отец
word #31/3576 is ваш
word #32/3576 is хотеть
word #33/3576 is такой
word #34/3576 is катя
word #35/3576 is евгений
word #36/3576 is фенёчек
word #37/3576 is день
word #38/3576 is слово
word #39/3576 is мой
word #40/3576 is время
word #41/3576 is весь
word #42/3576 is брат
w

In [224]:
#Если нужно будет найти самые частые слова

# def most_common(text):
#     while '»' in text:
#         text = text.replace('»', '')
#     while '«' in text:
#         text = text.replace('«', '')
#     while '(' in text:
#         text = text.replace('(', '')
#     while ')' in text:
#         text = text.replace(')', '')

#     symbs = ['—', "»", '«', 'note', '…']
#     stop = set(stopwords.words('russian') + list(string.punctuation) + symbs)

#     words = []

#     tokens = [i for i in tokenizer.tokenize(text.lower()) if i not in stop]
#     for word in tokens:
#         lemma = morph.parse(word)[0].normal_form
#         words.append(lemma)

#     counter = Counter(words)
#     most_common_100 = counter.most_common(100)
#     most_common_100 = most_common_100[0:100]

#     common = []
#     for el in most_common_100:
#         common.append(el[0])

#     return common

In [187]:
def analogies(word):
    similar = model.wv.most_similar(word)[0][0]
    different = model.wv.most_similar(negative = word)[0][0]
    second_word = model.wv.most_similar(positive = [word, similar], negative = different)[0][0]

    analogy = random.choice(model.wv.index_to_key)
    second_analogy = model.wv.most_similar(positive = [analogy, similar], negative = different)[0][0]

    return f"От слова '{word}' до слова '{second_word}' вектор примерно такой же как от слова '{analogy}' до слова '{second_analogy}'"

In [225]:
for i in range(101):
    word = random.choice(model.wv.index_to_key)
    print(analogies(word))

От слова 'герой' до слова 'дойти' вектор примерно такой же как от слова 'увлекаться' до слова 'почему'
От слова 'путь' до слова 'черта' вектор примерно такой же как от слова 'подобный' до слова 'заговорить'
От слова 'необходимо' до слова 'крепкий' вектор примерно такой же как от слова 'неудовольствие' до слова 'намерение'
От слова 'предварить' до слова 'ка' вектор примерно такой же как от слова 'шутя' до слова 'мягкий'
От слова 'игра' до слова 'заговорить' вектор примерно такой же как от слова 'плоский' до слова 'заговорить'
От слова 'разбитый' до слова 'возразить' вектор примерно такой же как от слова 'косынка' до слова 'оно'
От слова 'наставать' до слова 'катерина' вектор примерно такой же как от слова 'обращаться' до слова 'катерина'
От слова 'обходиться' до слова 'час' вектор примерно такой же как от слова 'приезжать' до слова 'немного'
От слова 'действительно' до слова 'дойти' вектор примерно такой же как от слова 'деревня' до слова 'сестра'
От слова 'поужинать' до слова 'уважение